In [2]:
import os
from supabase import create_client, Client
from typing import Dict, Optional, Tuple
import time

class SupabaseAuth:
    def __init__(self, url: str, key: str):
        self.supabase: Client = create_client(url, key)
    
    def create_user(self, email: str, password: str, user_metadata: Optional[Dict] = None) -> Dict:
        try:
            data = {
                "email": email,
                "password": password,
                "options": {
                    "data": user_metadata or {},
                    "email_confirm": False,  # Disable email confirmation
                }
            }
            
            response = self.supabase.auth.sign_up(data)
            return {
                "status": "success",
                "message": "OTP sent to email",
                "user": response.user
            }
        except Exception as e:
            return {"status": "error", "message": str(e)}
    
    def authenticate_user(self, email: str, password: str) -> Dict:
        try:
            response = self.supabase.auth.sign_in_with_password({
                "email": email,
                "password": password
            })
            return {"status": "success", "data": response}
        except Exception as e:
            return {"status": "error", "message": str(e)}
    
    def validate_user(self, access_token: Optional[str] = None) -> Dict:
        try:
            user = self.supabase.auth.get_user(access_token) if access_token else self.supabase.auth.get_user()
            return {"status": "success", "data": user}
        except Exception as e:
            return {"status": "error", "message": str(e)}
    
    def sign_out(self) -> Dict:
        try:
            self.supabase.auth.sign_out()
            return {"status": "success", "message": "Signed out successfully"}
        except Exception as e:
            return {"status": "error", "message": str(e)}


In [3]:
from dotenv import load_dotenv

load_dotenv()

auth = SupabaseAuth(
    url=os.getenv("SUPABASE_URL"),
    key=os.getenv("SUPABASE_KEY")
)

# Create a new user
new_user = auth.create_user(
    email="rishubcheddlla@gmail.com",
    password="secure_password",
    user_metadata={"name": "Rishub C R"}
)


In [4]:
new_user

{'status': 'success',
 'message': 'OTP sent to email',
 'user': User(id='e1601828-4162-4a53-92ef-35e01ec30580', app_metadata={'provider': 'email', 'providers': ['email']}, user_metadata={'name': 'Rishub C R'}, aud='authenticated', confirmation_sent_at=datetime.datetime(2024, 12, 12, 12, 1, 4, 435002, tzinfo=TzInfo(UTC)), recovery_sent_at=datetime.datetime(2024, 12, 12, 11, 48, 13, 272263, tzinfo=TzInfo(UTC)), email_change_sent_at=None, new_email=None, new_phone=None, invited_at=None, action_link=None, email='rishubcheddlla@gmail.com', phone='', created_at=datetime.datetime(2024, 12, 12, 12, 1, 4, 435002, tzinfo=TzInfo(UTC)), confirmed_at=None, email_confirmed_at=None, phone_confirmed_at=None, last_sign_in_at=None, role='', updated_at=datetime.datetime(2024, 12, 12, 12, 1, 4, 435002, tzinfo=TzInfo(UTC)), identities=[], is_anonymous=False, factors=None)}

In [5]:
# Authenticate a user
auth_response = auth.authenticate_user(
    email="rishubcheddlla@gmail.com",
    password="secure_password"
)

In [7]:
auth_response['data'].user

User(id='1d4a1aec-faff-4dc6-9abc-8d103c7a3c5e', app_metadata={'provider': 'email', 'providers': ['email']}, user_metadata={'email': 'rishubcheddlla@gmail.com', 'email_verified': False, 'name': 'Rishub C R', 'phone_verified': False, 'sub': '1d4a1aec-faff-4dc6-9abc-8d103c7a3c5e'}, aud='authenticated', confirmation_sent_at=datetime.datetime(2024, 12, 12, 11, 41, 57, 551991, tzinfo=TzInfo(UTC)), recovery_sent_at=datetime.datetime(2024, 12, 12, 11, 48, 13, 272263, tzinfo=TzInfo(UTC)), email_change_sent_at=None, new_email=None, new_phone=None, invited_at=None, action_link=None, email='rishubcheddlla@gmail.com', phone='', created_at=datetime.datetime(2024, 12, 12, 11, 34, 30, 326965, tzinfo=TzInfo(UTC)), confirmed_at=datetime.datetime(2024, 12, 12, 11, 42, 15, 227982, tzinfo=TzInfo(UTC)), email_confirmed_at=datetime.datetime(2024, 12, 12, 11, 42, 15, 227982, tzinfo=TzInfo(UTC)), phone_confirmed_at=None, last_sign_in_at=datetime.datetime(2024, 12, 12, 12, 1, 5, 92007, tzinfo=TzInfo(UTC)), role

In [41]:
# Validate a user
user_data = auth.validate_user()

In [42]:
user_data

{'status': 'success',
 'data': UserResponse(user=User(id='1d4a1aec-faff-4dc6-9abc-8d103c7a3c5e', app_metadata={'provider': 'email', 'providers': ['email']}, user_metadata={'email': 'rishubcheddlla@gmail.com', 'email_verified': False, 'name': 'Rishub C R', 'phone_verified': False, 'sub': '1d4a1aec-faff-4dc6-9abc-8d103c7a3c5e'}, aud='authenticated', confirmation_sent_at=datetime.datetime(2024, 12, 12, 11, 41, 57, 551991, tzinfo=TzInfo(UTC)), recovery_sent_at=datetime.datetime(2024, 12, 12, 11, 44, 26, 568406, tzinfo=TzInfo(UTC)), email_change_sent_at=None, new_email=None, new_phone=None, invited_at=None, action_link=None, email='rishubcheddlla@gmail.com', phone='', created_at=datetime.datetime(2024, 12, 12, 11, 34, 30, 326965, tzinfo=TzInfo(UTC)), confirmed_at=datetime.datetime(2024, 12, 12, 11, 42, 15, 227982, tzinfo=TzInfo(UTC)), email_confirmed_at=datetime.datetime(2024, 12, 12, 11, 42, 15, 227982, tzinfo=TzInfo(UTC)), phone_confirmed_at=None, last_sign_in_at=datetime.datetime(2024, 1

In [43]:
otp_response = auth.send_otp("rishubcheddlla@gmail.com")

In [44]:
otp_response

{'status': 'success', 'message': 'OTP sent successfully'}

In [45]:
verify_response = auth.verify_otp(
    email="rishubcheddlla@gmail.com",
    token="123456"  # OTP received by user
)

In [46]:
verify_response

{'status': 'error', 'message': 'Token has expired or is invalid'}